In [32]:
from extract_data import export_data
import seaborn as sns
import pandas as pd
import numpy as np
import p
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

import pickle
# import optuna
# import xgboost as xgb
# from optuna.samplers import TPESampler

In [33]:
data = pd.read_parquet("board-games-cleaned.parquet")
data.head()

,maxplayers,minage,minplayers,name,yearpublished,total,average,averageweight,bayesaverage,owned,usersrated,category,mechanics,decades
index,,,,,,,,,,,,,,
0,5,14,3,Die Macher,1986,6.0,7.66508,4.3477,7.29168,5251,4498,"b'[""Economic"", ""Negotiation"", ""Political""]'","b'[""Area Control / Area Influence"", ""Auction/B...",1983
1,4,12,3,Dragonmaster,1981,7.0,6.60815,1.9423,5.87150,1053,478,"b'[""Card Game"", ""Fantasy""]'","b'[""Trick-taking""]'",1983
2,4,10,2,Samurai,1998,6.0,7.44119,2.5085,7.28295,11870,12019,"b'[""Abstract Strategy"", ""Medieval""]'","b'[""Area Control / Area Influence"", ""Hand Mana...",2002
3,4,12,2,Tal der Könige,1992,5.0,6.60675,2.6667,5.76636,523,314,"b'[""Ancient""]'","b'[""Action Point Allowance System"", ""Area Cont...",1992
4,6,12,3,Acquire,1964,6.0,7.35830,2.5089,7.21895,18682,15195,"b'[""Economic""]'","b'[""Hand Management"", ""Stock Holding"", ""Tile P...",1964


In [34]:
data.drop(columns=["category","mechanics","name"],inplace=True)
data.isnull().sum()

maxplayers       0
minage           0
minplayers       0
yearpublished    0
total            0
average          0
averageweight    0
bayesaverage     0
owned            0
usersrated       0
decades          0
dtype: int64

In [35]:
X = data.drop(columns=["average"])
y = data["average"]

In [36]:
#train_test_valid spliting
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=2409,train_size=0.7)
X_test,X_valid,y_test,y_valid = train_test_split(X_test,y_test,random_state=2409,test_size=0.5)
print("train_size:",round(len(X_train)/len(X),2))
print("test_size:",round(len(X_test)/len(X),2))
print("train_size:",round(len(X_valid)/len(X),2))

train_size: 0.7
test_size: 0.15
train_size: 0.15


In [37]:
#simple xgboost
xg = XGBRegressor()
xg.fit(X_train,y_train)
rmse_test_xg_simple = mean_squared_error(xg.predict(X_test) ,y_test)
print(rmse_test_xg_simple)

0.7325464325317161


In [38]:
#Initialize domain space for range of values
# hp.choice(label, options) — Returns one of the options, which should be a list or tuple.
# hp.randint(label, upper) — Returns a random integer between the range [0, upper).
# hp.uniform(label, low, high) — Returns a value uniformly between low and high.
# hp.quniform(label, low, high, q) — Returns a value round(uniform(low, high) / q) * q, i.e it rounds the decimal values and returns an integer.
# hp.normal(label, mean, std) — Returns a real value that’s normally-distributed with mean and standard deviation sigma.

space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

In [39]:
#Define objective function 
def objective(space):
    reg=XGBRegressor(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_train, y_train), ( X_valid, y_valid)]
    
    reg.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = reg.predict(X_valid)
    rmse = mean_squared_error(y_valid, pred)
#     print ("SCORE:", rmse)
    return (rmse)

In [40]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/

[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/

C:\Users\nerses.aghababyan\Anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\nerses.aghababyan\Anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\nerses.aghababyan\Anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\nerses.aghababyan\Anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_round

[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/

[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:09] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/

...

[19:36:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/

C:\Users\nerses.aghababyan\Anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\nerses.aghababyan\Anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\nerses.aghababyan\Anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\nerses.aghababyan\Anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_round

[19:36:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/

[19:36:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/metric/auc.cc:328: Dataset is empty, or contains only positive or negative samples.
[19:36:17] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/

C:\Users\nerses.aghababyan\Anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\nerses.aghababyan\Anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\nerses.aghababyan\Anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\nerses.aghababyan\Anaconda3\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_round

In [41]:
best_hyperparams

{'colsample_bytree': 0.753847982759177,
 'gamma': 5.228993762656932,
 'max_depth': 4.0,
 'min_child_weight': 7.0,
 'reg_alpha': 40.0,
 'reg_lambda': 0.9616061102216716}

In [42]:
best_xg = XGBRegressor(colsample_bytree=0.596593434466355,gamma=4.238336169318275,
                    max_depth=15,min_child_weight=1.0,reg_alpha=40,reg_lambda=0.5448000648106297)
best_xg.fit(X_train,y_train)
rmse_test_xg = mean_squared_error(best_xg.predict(X_test),y_test)
print("tuned",rmse_test_xg)
print("simple:",rmse_test_xg_simple)

tuned 0.6985464714564031
simple: 0.7325464325317161


In [43]:
#simple rf
rf=RandomForestRegressor()
rf.fit(X_train,y_train)
rmse_test_rf_simple = mean_squared_error(rf.predict(X_test),y_test)
print(rmse_test_rf_simple)

0.7311332307099827


In [44]:
#random_forest params tuning
seed=2
def objective(params):
    est=int(params['n_estimators'])
    md=int(params['max_depth'])
    msl=int(params['min_samples_leaf'])
    mss=int(params['min_samples_split'])
    model=RandomForestRegressor(n_estimators=est,max_depth=md,min_samples_leaf=msl,min_samples_split=mss)
    model.fit(X_train,y_train)
    pred=model.predict(X_valid)
    score=mean_squared_error(y_valid,pred)
    return score

def optimize(trial):
    params={'n_estimators':hp.uniform('n_estimators',100,500),
           'max_depth':hp.uniform('max_depth',5,20),
           'min_samples_leaf':hp.uniform('min_samples_leaf',1,5),
           'min_samples_split':hp.uniform('min_samples_split',2,6)}
    best=fmin(fn=objective,space=params,algo=tpe.suggest,trials=trial,max_evals=100)
    return best

trial=Trials()
best=optimize(trial)

100%|██████████████████████████████████████████████| 100/100 [15:07<00:00,  9.07s/trial, best loss: 0.6727684733385931]


In [45]:
best

{'max_depth': 13.088669786520345,
 'min_samples_leaf': 3.270748431257397,
 'min_samples_split': 3.7537533125854146,
 'n_estimators': 343.31196834838863}

In [46]:
#best_rf
best_rf = RandomForestRegressor(max_depth=12.953930139655872,min_samples_leaf=5,
                               min_samples_split=3,n_estimators=190)
best_rf.fit(X_train,y_train)
rmse_test_rf = mean_squared_error(best_rf.predict(X_test),y_test)
print("tuned",rmse_test_rf)
print("simple",rmse_test_rf_simple)

tuned 0.6927236187216353
simple 0.7311332307099827


In [47]:
TID=[t['tid'] for t in trial.trials]
Loss=[t['result']['loss'] for t in trial.trials]
maxd=[t['misc']['vals']['max_depth'][0] for t in trial.trials]
nest=[t['misc']['vals']['n_estimators'][0] for t in trial.trials]
min_ss=[t['misc']['vals']['min_samples_split'][0] for t in trial.trials]
min_sl=[t['misc']['vals']['min_samples_leaf'][0] for t in trial.trials]

hyperopt_rfr=pd.DataFrame({'tid':TID,'loss':Loss,
                          'max_depth':maxd,'n_estimators':nest,
                          'min_samples_split':min_ss, 'min_samples_leaf':min_sl})

In [48]:
plt.subplots(3,2,figsize=(10,10))
plt.subplot(3,2,1)
sns.scatterplot(x='tid',y='max_depth',data=hyperopt_rfr)
plt.subplot(3,2,2)
sns.scatterplot(x='tid',y='loss',data=hyperopt_rfr)
plt.subplot(3,2,3)
sns.scatterplot(x='tid',y='n_estimators',data=hyperopt_rfr)
plt.subplot(3,2,4)
sns.scatterplot(x='tid',y='min_samples_leaf',data=hyperopt_rfr)
plt.subplot(3,2,5)
sns.scatterplot(x='tid',y='min_samples_split',data=hyperopt_rfr)

plt.tight_layout()

NameError: name 'plt' is not defined

In [28]:
 with open("xgb", 'wb') as handle:
            pickle.dump(best_xg, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
 with open("rf", 'wb') as handle:
            pickle.dump(best_rf, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [31]:
#load_model
#  with open("rf", 'rb') as handle:
#             model = pickle.load(handle) 